`Note: Code chunks with (R) need to be run in order for the dataset to be processed and analyzed properly.`

# Libraries

In [1]:
# (R) Load required modules
import datetime as dt
import numpy as np
import pandas as pd

# Plotting modules
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FixedLocator, FuncFormatter

import seaborn as sns

# Market basket analysis
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Load Data

In [2]:
# (R) Load dataset
df = pd.read_csv("../dataset/supermarket/fp.csv")
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].apply(lambda x: x.year)

pricing = pd.read_csv("../dataset/supermarket/fp_price.csv")
pricing = pricing.rename(columns={"product_title": "product"})

In [11]:
# FairPrice color scheme
# Blue: #0C3578
# Red: #ED2E24

# Data Visualization

In [55]:
# Reset matplotlib rcParams
matplotlib.rc_file_defaults()

In [56]:
# Create data frame of monthly sales for all products.
sales_day = df.groupby("date", as_index=False).agg(daily_sales=("total_sales", "sum"))
sales_day = sales_day.set_index("date")
sales_month = sales_day["daily_sales"].resample("M").sum()
sales_month = sales_month.reset_index()

In [ ]:
# Plot of sales over time.
sns.set_theme(rc={"figure.dpi":300, 'savefig.dpi':300},
              style="white")   # adjust image resolution
sns.set(rc={"figure.facecolor": "#F8F8F8"})
ax = sns.lineplot(data=sales_month, 
                  x="date", 
                  y="daily_sales",
                  color="#0C3578",
                  linewidth=2);

# Setting x-axis tick labels
ax_xaxis = sales_month.date[::6].to_list()
ax_xaxis.append(sales_month.date[23])
ax.set_xticks(ax_xaxis)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))

# Setting y-axis tick labels
ax.yaxis.set_major_locator(FixedLocator([3000, 6000, 9000, 12000, 15000, 18000]))
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: "${:,}".format(x)))

# Setting fonts for tick labels
ax.set_xticklabels(ax.get_xticklabels(), family="serif")
ax.set_yticklabels(ax.get_yticklabels(), family="serif")

# Setting y-axis label and y-axis parameters
# ax.set_ylabel("Total Monthly Sales", family="serif")
ax.set_ylim(0, 20000)
ax.tick_params(left=False)

ax.set_title("Monthly Total Sales", family="serif", weight="bold", size=16)

# Remove x-axis label
ax.xaxis.label.set_visible(False)
ax.yaxis.label.set_visible(False)

# Removing spines
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.spines["bottom"].set_color("#CBCACA")

# Adding gridlines in the background
ax.xaxis.grid(True, which="major", linestyle="--", alpha=0.05, color="#0C3578")
ax.yaxis.grid(True, which="major", linestyle="--", alpha=0.05, color="#0C3578")

# Add rectangles to highlight certain periods
ax.axvspan(sales_month["date"][8], sales_month["date"][11], color="#ED2E24", alpha=0.2)
ax.axvspan(sales_month["date"][20], sales_month["date"][23], color="#ED2E24", alpha=0.2);

ax.annotate("Oct-Dec", (sales_month["date"][8] + pd.DateOffset(5), 500), family="serif", size=9)
ax.annotate("Oct-Dec", (sales_month["date"][20] + pd.DateOffset(5), 500), family="serif", size=9)

ax.set_facecolor("#F8F8F8")

In [58]:
# Create data frame of monthly sales for top five products.
top_sales = (df.groupby("product", as_index=False)
               .agg(total_sales=("total_sales", "sum"))
               .sort_values("total_sales", ascending=False))
top5_products = top_sales["product"][0:5].to_list()

top5_df = df[df["product"].isin(top5_products)]
top5_sales_day = (top5_df.groupby(["product", "date"], as_index=False)
                         .agg(daily_sales=("total_sales", "sum")))
top5_sales_day = top5_sales_day.set_index("date")
top5_sales_month = top5_sales_day.groupby([pd.Grouper(freq="M"), "product"])
top5_sales_month = top5_sales_month["daily_sales"].sum().reset_index()

In [ ]:
# Plot of sales of top five products over time.
sns.set(rc={"figure.facecolor": "#F8F8F8"})

ax = sns.lineplot(data=top5_sales_month,
                  x="date",
                  y="daily_sales",
                  hue="product",
                  palette=["#0C3578", "#B68505", "#ED2E24", "#076C07", "#0370DB"],
                  linewidth=2)


ax_xaxis = list(top5_sales_month["date"].unique()[::6])
ax_xaxis.append(top5_sales_month["date"].unique()[23])
ax.set_xticks(ax_xaxis)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))

ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_color("#CBCACA")

ax.yaxis.label.set_visible(False)
ax.xaxis.label.set_visible(False)

ax.set_title("Monthly Sales of Top Five Products", family="serif", weight="bold", size=16)

ax.yaxis.set_major_locator(FixedLocator([500, 1000, 1500, 2000, 2500]))
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: "${:,}".format(x)))
ax.set_ylim(-100, 2600)

ax.set_xticklabels(ax.get_xticklabels(), family="serif")
ax.set_yticklabels(ax.get_yticklabels(), family="serif")

ax.xaxis.grid(True, which="major", linestyle="--", alpha=0.05, color="#0C3578")
ax.yaxis.grid(True, which="major", linestyle="--", alpha=0.05, color="#0C3578")

# Adjust the size of the legend
ax.get_legend().remove()

# Annotate text at end of line plot

date_loc = top5_sales_month["date"].unique()[23] + np.timedelta64(10,'D')


ax.annotate("Jing Li Hwa Frozen Fish Ball - Roe", xy=(date_loc, 2560.00),
            color="#0370DB", size=8, family="serif");
ax.annotate("FairPrice Thai Hom Mali Superior \nFragrant Rice", xy=(date_loc, 2360.70),
            color="#0C3578", size=8, family="serif");
ax.annotate("Maling Premium Luncheon Meat -\nPork", xy=(date_loc, 2185.75),
            color="#076C07", size=8, family="serif");
ax.annotate("Milo Chocolate Malt Drink Powder \nwith Milk - Australian Recipe", xy=(date_loc, 1558.34),
            color="#B68505", size=8, family="serif");
ax.annotate("Lee Kum Kee Oyster Sauce - \nPanda Brand (770g)", xy=(date_loc, 1337.20),
            color="#ED2E24", size=8, family="serif");


ax.set_facecolor("#F8F8F8")

In [60]:
# Create top twenty products in sales with percentage contribution calculated.

total_sales = top_sales["total_sales"].sum()

perc_total_sales = (top_sales["total_sales"] / total_sales * 100).values
top_sales["perc_total_sales"] = np.round(perc_total_sales, 1)
top_sales["perc_total_sales"] = top_sales["perc_total_sales"].astype("str")
top_sales["perc_total_sales"] = top_sales["perc_total_sales"] + "%"

top20_sales = top_sales.head(20)

In [ ]:
# Plot barplot of top 20 products. 
sns.set(rc={"figure.facecolor": "#F8F8F8"})

ax = sns.barplot(data=top20_sales,
                 x="total_sales",
                 y="product",
                 palette=list(["#0C3578"]*5 + ["#848484"]*15));


ax.xaxis.label.set_visible(False);
ax.yaxis.label.set_visible(False);

ax.set_title("Total Sales Contribution by Products", family="serif", size=16, weight="bold", pad=15, horizontalalignment="right")
ax.set_facecolor("#F8F8F8")
ax.spines["top"].set_color("#CBCACA")
ax.spines["bottom"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

ax.xaxis.grid(True, which="major", alpha=0.05, color="#0C3578")
ax.xaxis.set_label_position("top")
ax.set_xticklabels(ax.get_xticks(), family="serif")
ax.set_yticklabels(ax.get_yticklabels(), family="serif", size=9)
ax.xaxis.tick_top()
ax.tick_params(axis='x', which='major', pad=-2)
ax.tick_params(top=False)
ax.xaxis.set_major_locator(FixedLocator([3000, 6000, 9000, 12000, 15000]))
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: "${:,}".format(x)))

for i in range(5):
    ax.get_yticklabels()[i].set_color("#0C3578")
    ax.get_yticklabels()[i].set_weight("bold")

# To add sales percentage onto the barplots    
for patch, perc in zip(ax.patches, top20_sales["perc_total_sales"]):
    ax.annotate(perc, 
                (patch.get_width(), patch.get_y()),
                ha="left", va="top",
                xytext=(patch.get_width()+100,patch.get_y()+0.13),
                size=8.5,
                family="serif")
    
ax.set_facecolor("#F8F8F8")

# Customer Segmentation

In [8]:
# (R) Create the RFM values for customer segmentation
def rfm(dataset):
    recent_date = dataset["date"].max()
    
    cid = (dataset.groupby("customer_id", as_index=False)
                  .agg(recent_purchase=("date", "max"),
                       frequency=("receipt_num", pd.Series.nunique),
                       monetary=("total_sales", "sum"))
                  .sort_values("frequency", ascending=False))
    cid["recency"] = cid["recent_purchase"].apply(lambda day: (recent_date-day).days)
    cid = cid.drop("recent_purchase", axis=1)
    
    return cid

cid = rfm(df)

# Convert RFM values to RFM categories
cid["r_rank"] = pd.qcut(cid["recency"], q=3, labels=["1", "2", "3"])
cid["f_rank"] = pd.cut(cid["frequency"], bins=[0, 1, 10, 83], labels=["3", "2", "1"])
cid["m_rank"] = pd.qcut(cid["monetary"], q=3, labels=["3", "2", "1"])

cid["rfm"] = cid[["r_rank", "f_rank", "m_rank"]].apply(lambda row: "".join(row.values), axis=1)

#### RFM
* Top customers = 111:                              High recent, high freq, high money
* Loyal = 112, 113, 212, 213, 211, 311, 312, 313:   High/mid/low recent, high freq, high/mid/low money
* Promising = 121, 122, 123, 221, 222, 223:         High/Mid recent, mid freq, high/mid/low money
* Slipped promising = 321, 322, 323:                Low recent, mid freq, high/mid/low money
* New = 131, 132, 133:                              High recent, low freq, high/mid/low money
* One-time Purchase = 231, 232, 233, 331, 332, 333: Mid/low recent, low freq, high/mid/low money

In [9]:
# (R) Classify customers into segments
conditions = [
    cid["rfm"].isin(["111"]),
    cid["rfm"].isin(["112", "113", "211", "212", "213", "311", "312", "313"]),
    cid["rfm"].isin(["121", "122", "123", "221", "222", "223"]),
    cid["rfm"].isin(["321", "322", "323"]),
    cid["rfm"].isin(["131", "132", "133"]),
    cid["rfm"].isin(["231", "232", "233", "331", "332", "333"])
]
replacements = ["Top Priority", "Loyal", "Promising", "Slipped Promising", "New Arrivals", "One-time Purchase"]
cid["cust_segm"] = np.select(conditions, replacements, default=np.nan)

In [664]:
# Data frame for frequency histogram
freq_df = cid["frequency"].value_counts().reset_index()
freq_df = freq_df.rename(columns={"index": "n_transac"})


In [ ]:
# Plot of frequency histogram
sns.set(rc={"figure.facecolor": "#F8F8F8"})

ax = sns.barplot(data=freq_df, x="n_transac", y="frequency", color="#0C3578")
ax.set_xlabel("Number of Transactions by Customer", family="serif")
ax.yaxis.label.set_visible(False)
# ax.set_ylabel("Frequency Count", family="serif")

ax.set_xticklabels(ax.get_xticklabels(), family="serif")
ax.set_yticklabels(ax.get_yticklabels(), family="serif")

ax.set_title("Customer Count by Transaction Frequency", family="serif", size=16, weight="bold")

# ax.xaxis.grid(True, which='major', linestyle="--", alpha=0.2)
ax.yaxis.grid(True, which='major', alpha=0.05, color="#0C3578")

ax.spines["bottom"].set_color("#CBCACA")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

ax.tick_params(bottom=True, color="#CBCACA")

ax.set_facecolor("#F8F8F8")

In [ ]:
# Plot of monetary value histogram cut at $300
sns.set(rc={"figure.facecolor": "#F8F8F8"})

ax = sns.histplot(data=cid, x="monetary", color="#18376A", alpha=1)


ax.set_title("Customer Count by Transaction Value", family="serif", size=16, weight="bold")
ax.set_xlabel("Transaction Value", family="serif")
ax.yaxis.label.set_visible(False)

ax.set_xticklabels(ax.get_xticklabels(), family="serif")
ax.set_yticklabels(ax.get_yticklabels(), family="serif")

ax.xaxis.set_major_locator(FixedLocator([50, 100, 150, 200, 250, 300]))
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: "${:,}".format(x)))

ax.set_xlim(0, 300);
ax.spines["bottom"].set_color("#CBCACA")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

#ax.tick_params(bottom=True, color="#CBCACA")
ax.tick_params(bottom=False)

ax.xaxis.grid(False)
ax.yaxis.grid(True, which="major", alpha=0.05, color="#0C3578")

ax.set_facecolor("#F8F8F8")

# 13 customers bought > $300 in total value
# cid[cid["monetary"] > 300]


In [ ]:
# Plot of recency value histogram
sns.set(rc={"figure.facecolor": "#F8F8F8"})

ax = sns.histplot(data=cid, x="recency", color="#18376A", alpha=1)

ax.set_title("Customer Count by Days Since Last Purchase", family="serif", size=16, weight="bold")
ax.set_xlabel("Number of Days Since Last Purchase", family="serif")
ax.yaxis.label.set_visible(False)

ax.set_xticklabels(ax.get_xticklabels(), family="serif")
ax.set_yticklabels(ax.get_yticklabels(), family="serif")

#ax.xaxis.set_major_locator(FixedLocator([50, 100, 150, 200, 250, 300]))
#ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: "${:,}".format(x)))

ax.spines["bottom"].set_color("#CBCACA")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

ax.xaxis.grid(False)
ax.yaxis.grid(True, which="major", alpha=0.05, color="#0C3578")

ax.set_facecolor("#F8F8F8")

In [10]:
# (R) Reclassify customers into three bigger segments
conditions2 = [
    cid["cust_segm"].isin(["Top Priority", "Loyal"]),
    cid["cust_segm"].isin(["Promising", "Slipped Promising"]),
    cid["cust_segm"].isin(["New Arrivals", "One-time Purchase"]),
]
replacements2 = ["grp1", "grp2", "grp3"]
cid["cust_segm2"] = np.select(conditions2, replacements2, default=np.nan)
cid["cust_segm2"] = cid["cust_segm2"].astype("category")
cid["cust_segm2"] = cid["cust_segm2"].cat.set_categories(replacements2, ordered=True)

# Merge customer segments into original df with individual product titles
df = df.merge(cid, on="customer_id", how="left")

In [ ]:
# Data frame for barplot of customer count in each customer segment
cust_segm_df = cid["cust_segm"].value_counts().reset_index()
cust_segm_df = cust_segm_df.rename(columns={"cust_segm": "cust_count",
                                            "index": "cust_segm"})
cust_segm_df["cust_segm"] = cust_segm_df["cust_segm"].astype("category")
cust_segm_df["cust_segm"] = cust_segm_df["cust_segm"].cat.set_categories(replacements, ordered=True)

total_cust = cust_segm_df["cust_count"].sum()

perc_cust_count = (cust_segm_df["cust_count"] / total_cust * 100).values
cust_segm_df["perc_cust_count"] = np.round(perc_cust_count, 1)
cust_segm_df["perc_cust_count"] = cust_segm_df["perc_cust_count"].astype("str")
cust_segm_df["perc_cust_count"] = cust_segm_df["perc_cust_count"] + "%"

In [ ]:
# Barplot for customer count in each customer segment
sns.set(rc={"figure.facecolor": "#F8F8F8",
            "figure.figsize": (6, 2)})

ax = sns.barplot(data=cust_segm_df, x="cust_count", y="cust_segm", 
                 palette=list(["#0C3578"]*2 + ["#848484"]*4)) # Color change


ax.xaxis.label.set_visible(False);
ax.yaxis.label.set_visible(False);

ax.set_title("Customer Count in Each Segment", family="serif", size=16, weight="bold", horizontalalignment="center")
ax.set_facecolor("#F8F8F8")
# ax.spines["top"].set_color("#CBCACA")
ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_color("#CBCACA")
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

ax.xaxis.grid(True, which="major", alpha=0.05, color="#0C3578")
# ax.xaxis.set_label_position("top")
ax.set_xticklabels(ax.get_xticks(), family="serif", size=8)
ax.set_yticklabels(ax.get_yticklabels(), family="serif", size=9)
# ax.xaxis.tick_top()
# ax.tick_params(axis='x', which='major', pad=-2)
ax.tick_params(bottom=False)
ax.xaxis.set_major_locator(FixedLocator([500, 1000, 1500, 2000, 2500, 3000, 3500]))
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: "{:,}".format(x)))

ax.set_xlim(0, 4000)

# To add sales percentage onto the barplots
for patch, perc in zip(ax.patches, ["0.4%", "0.1%", "21.8%", "6.1%", "20.0%", "51.7%"]):
    ax.annotate(perc, 
                (patch.get_width(), patch.get_y()),
                ha="left",
                xytext=(patch.get_width()+20,patch.get_y()+0.5),
                size=8.5,
                family="serif")
    
ax.set_facecolor("#F8F8F8")

In [ ]:
# (Priority) Barplot for individual customer segment for impact estimation customer count in each customer segment
sns.set(rc={"figure.facecolor": "#F8F8F8",
            "figure.figsize": (6, 2)})

ax = sns.barplot(data=cust_segm_df, x="cust_count", y="cust_segm", 
                 palette=list(["#0C3578"]*2 + ["#C3C3C3"]*4))


ax.xaxis.label.set_visible(False);
ax.yaxis.label.set_visible(False);

ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

ax.xaxis.grid(True, which="major", alpha=0.05, color="#0C3578")
# ax.xaxis.set_label_position("top")
ax.set_xticklabels([])
ax.tick_params(bottom=False)
# ax.set_xticklabels(ax.get_xticks(), family="serif", size=8)
ax.set_yticklabels(ax.get_yticklabels(), family="serif", size=9)
# ax.xaxis.tick_top()
# ax.tick_params(axis='x', which='major', pad=-2)
ax.tick_params(bottom=False)
#ax.xaxis.set_major_locator(FixedLocator([500, 1000, 1500, 2000, 2500, 3000, 3500]))
#ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: "{:,}".format(x)))

ax.set_xlim(0, 4000)
ax.xaxis.grid(False)

for i in range(2):
    ax.get_yticklabels()[i].set_color("#0C3578")
    ax.get_yticklabels()[i].set_weight("bold")

for i in [2, 3, 4, 5]:
    ax.get_yticklabels()[i].set_color("#C3C3C3")

# To add sales percentage onto the barplots
for patch, perc, col, weight in zip(ax.patches, 
                       ["23", "5", "407", "1302", "1432", "3424"],
                       ["#0C3578", "#0C3578", "#C3C3C3", "#C3C3C3", "#C3C3C3", "#C3C3C3"],
                       ["bold"]*2 + ["normal"]*4):
    ax.annotate(perc, 
                (patch.get_width(), patch.get_y()),
                ha="left",
                xytext=(patch.get_width()+20,patch.get_y()+0.55),
                size=8.5,
                family="serif",
                color=col,
                weight=weight)

ax.set_facecolor("#F8F8F8")

In [ ]:
# (Promising) Barplot for individual customer segment for impact estimation customer count in each customer segment
sns.set(rc={"figure.facecolor": "#F8F8F8",
            "figure.figsize": (6, 2)})

ax = sns.barplot(data=cust_segm_df, x="cust_count", y="cust_segm", 
                 # Change the palette color
                 palette=list(["#C3C3C3"]*2 + ["#0C3578"]*2 + ["#C3C3C3"]*2))

ax.xaxis.label.set_visible(False);
ax.yaxis.label.set_visible(False);

ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

ax.xaxis.grid(True, which="major", alpha=0.05, color="#0C3578")

ax.set_xticklabels([])
ax.tick_params(bottom=False)

ax.set_yticklabels(ax.get_yticklabels(), family="serif", size=9)

ax.tick_params(bottom=False)

ax.set_xlim(0, 4000)
ax.xaxis.grid(False)

for i in [2, 3]:
    ax.get_yticklabels()[i].set_color("#0C3578")
    ax.get_yticklabels()[i].set_weight("bold")

for i in [0, 1, 4, 5]:
    ax.get_yticklabels()[i].set_color("#C3C3C3")

# To add sales percentage onto the barplots
for patch, perc, col, weight in zip(ax.patches, 
                       ["23", "5", "407", "1302", "1432", "3424"],
                       ["#C3C3C3", "#C3C3C3", "#0C3578", "#0C3578", "#C3C3C3", "#C3C3C3"],
                       ["normal"]*2 + ["bold"]*2 + ["normal"]*2):
    ax.annotate(perc, 
                (patch.get_width(), patch.get_y()),
                ha="left",
                xytext=(patch.get_width()+20,patch.get_y()+0.55),
                size=8.5,
                family="serif",
                color=col,
                weight=weight)

ax.set_facecolor("#F8F8F8")

In [ ]:
# (Potential) Barplot for individual customer segment for impact estimation customer count in each customer segment
sns.set(rc={"figure.facecolor": "#F8F8F8",
            "figure.figsize": (6, 2)})

ax = sns.barplot(data=cust_segm_df, x="cust_count", y="cust_segm", 
                 # Change the palette color
                 palette=list(["#C3C3C3"]*4 + ["#0C3578"]*2))

ax.xaxis.label.set_visible(False);
ax.yaxis.label.set_visible(False);

ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

ax.xaxis.grid(True, which="major", alpha=0.05, color="#0C3578")

ax.set_xticklabels([])
ax.tick_params(bottom=False)

ax.set_yticklabels(ax.get_yticklabels(), family="serif", size=9)

ax.set_xlim(0, 4000)
ax.xaxis.grid(False)

for i in [4, 5]:
    ax.get_yticklabels()[i].set_color("#0C3578")
    ax.get_yticklabels()[i].set_weight("bold")

for i in [0, 1, 2, 3]:
    ax.get_yticklabels()[i].set_color("#C3C3C3")

# To add sales percentage onto the barplots
for patch, perc, col, weight in zip(ax.patches, 
                       ["23", "5", "407", "1302", "1432", "3424"],
                       ["#C3C3C3", "#C3C3C3", "#C3C3C3", "#C3C3C3", "#0C3578", "#0C3578"],
                       ["normal"]*4 + ["bold"]*2):
    ax.annotate(perc, 
                (patch.get_width(), patch.get_y()),
                ha="left",
                xytext=(patch.get_width()+20,patch.get_y()+0.55),
                size=8.5,
                family="serif",
                color=col,
                weight=weight)

ax.set_facecolor("#F8F8F8")

# Product Bundling

In [11]:
# (R) Define function to flatten product list based on customer_id

# Function outputs dataset based on cust_segm2 column
def flatten_split(dataset, grp_by_col="customer_id"):
    
    # Convert flatten product titles from each customer and convert to list
    df_grpby = dataset.groupby(grp_by_col)["product"].apply(", ".join)
    df_grpby = df_grpby.apply(lambda x: x.split(", "))
    df_grpby = pd.DataFrame({grp_by_col: df_grpby.index,
                             "list_products": df_grpby.values})
    df_grpby[grp_by_col] = df_grpby[grp_by_col].apply(str)
    df_grpby = df_grpby.sort_values(grp_by_col)

    # Merge df with listed product titles to the original cid df
    # List of product lists are duplicated
    df_grpby = (df_grpby.merge(df, on=grp_by_col, how="left")
                        .drop_duplicates(grp_by_col))

    # Split customers into three groups for product bundling
    grp1 = df_grpby.query("cust_segm2 == 'grp1'")      # priority
    grp2 = df_grpby.query("cust_segm2 == 'grp2'")      # promising
    grp3 = df_grpby.query("cust_segm2 == 'grp3'")      # potential
    
    return grp1, grp2, grp3

In [12]:
# (R) Flatten product titles by customer_id, then split by customer segments
cgrp1, cgrp2, cgrp3 = flatten_split(df, grp_by_col="customer_id")

In [13]:
# (R) Define function to calculate the support metrics for the item(s)
def grp_assoc(dataframe, min_support=0.05, metric="confidence", min_threshold=0.4, sort_by="lift"):
    
    # One-hot encode the product titles
    one_hot_encoder = TransactionEncoder()
    one_hot_products = one_hot_encoder.fit_transform(dataframe["list_products"])
    
    # customer_id not present, but still in line at this point
    df_ptitles = pd.DataFrame(one_hot_products, columns=one_hot_encoder.columns_)
    
    # Calculate support for products
    apr_ptitles = apriori(df_ptitles, min_support=min_support, use_colnames=True)
    
    # Calculate remaining support, lift & confidence for products
    assoc_ptitles = association_rules(apr_ptitles, metric=metric, min_threshold=min_threshold)
    
    # Convert frozen set object to string for plotting 
    assoc_ptitles["antecedents"] = assoc_ptitles["antecedents"].apply(lambda x: list(x)[0])
    assoc_ptitles["consequents"] = assoc_ptitles["consequents"].apply(lambda x: list(x)[0])
    
    return assoc_ptitles.sort_values(sort_by, ascending=False)

In [14]:
# (R) Calculate the support metrics for each of the customer segments
crules1 = grp_assoc(cgrp1, min_support=0.25, metric="lift", min_threshold=1, sort_by="lift")
crules2 = grp_assoc(cgrp2, min_support=0.05, metric="lift", min_threshold=1, sort_by="lift")
crules3 = grp_assoc(cgrp3, min_support=0.015, metric="lift", min_threshold=1, sort_by="lift")

In [348]:
# Create date frame for heatmap plotting in the next step.
def df_heatmap(df_rules):
    
    # Convert frozen set object to string for plotting and pivot dataframe to wide format
    df_heatmap = (df_rules.sort_values("lift", ascending=False)
                          .head(20)
                          .pivot("antecedents", "consequents", "lift"))
    
    return df_heatmap

# Create data frame for heatmap plotting for each customer segment
cgrp1_heatmap = df_heatmap(crules1)
cgrp2_heatmap = df_heatmap(crules2)
cgrp3_heatmap = df_heatmap(crules3)

In [ ]:
# Heatmap for Promising Customers Segment 2.
sns.set(rc={"figure.facecolor": "#F8F8F8",
            "figure.figsize": (6, 4)})

ax = sns.heatmap(data=cgrp2_heatmap, annot=True, linewidths=0.3, annot_kws={"family":"serif"}, cbar=False)

ax.xaxis.label.set_visible(False)
ax.yaxis.label.set_visible(False)

# ax.set_title("Lift Score for Promising Customers", family="serif", weight="bold", size=20, pad=15)

ax.set_xticklabels(ax.get_xticklabels(), family="serif")
ax.set_yticklabels(ax.get_yticklabels(), family="serif");

for i in [7, 8, 9]:
    ax.get_yticklabels()[i].set_color("#0C3578")
    ax.get_yticklabels()[i].set_weight("bold")


# Trip Driving

In [15]:
# (R) Assigning new dataframes based on customer segments
df_grp1 = df[df["cust_segm2"] == "grp1"]
df_grp2 = df[df["cust_segm2"] == "grp2"]
df_grp3 = df[df["cust_segm2"] == "grp3"]

In [16]:
# (R) Define trip driving calculation and function
def trip_driving(dataframe):

    # Count number of transactions each product is present in
    trans_count = (dataframe.groupby("product")
                            .agg(trans_num=("receipt_num", pd.Series.nunique))
                            .reset_index()
                            .sort_values("trans_num", ascending=False))

    # Create empty list to collect the unique product count
    unique_prod_count = []

    # Iterate through the unique products for each customer segment
    for prod in trans_count["product"]:
        
        # Obtain receipt_num that contains the product
        prod_receipt = dataframe[dataframe["product"] == prod]["receipt_num"]
        
        # Subset all orders that contain the product from the main dataframe
        prod_df = dataframe[dataframe["receipt_num"].isin(prod_receipt)]
        
        # Group the dataframe by receipt_num, then count the number of unique products in each transaction
        uni_prod_count = (prod_df.groupby("receipt_num")
                                 .agg(nunique_prod=("product", pd.Series.nunique)))
        
        # Sum the number of all unique product counts
        unique_prod_count.append(uni_prod_count["nunique_prod"].sum())
    
    trans_count["unique_prod_count"] = unique_prod_count
    trans_count["avg_share"] = trans_count["trans_num"] / trans_count["unique_prod_count"]
        
    # Count the number of each product in each transaction
    prod_count_by_trans = (dataframe.groupby(["receipt_num", "product"])
                                    .agg(receipt_prod_count=("product", pd.Series.nunique))
                                    .reset_index())
    prod_count_by_trans = prod_count_by_trans.merge(trans_count, on="product", how="left")
    
    # Create empty dictionary to collect the dependable product count
    count_dict = {}
    
    for group, frame in prod_count_by_trans.groupby("receipt_num"):
        
        # Identify highest avg_share product in the transaction
        max_share = frame["avg_share"].max()
        top_avg_df = frame[frame["avg_share"] == max_share]
        
        for i in range(top_avg_df.shape[0]):
            product_title = top_avg_df["product"].iloc[i]
            product_count = top_avg_df["receipt_prod_count"].iloc[i]
        
            # Add the product if absent; increase count of product if present
            if product_title in count_dict:
                count_dict[product_title] += product_count
            else:
                count_dict[product_title] = product_count
    
    # Merge all calculated dataframes together
    depend_df = pd.DataFrame({"product": list(count_dict.keys()),
                              "most_depend": list(count_dict.values())})
    depend_df = trans_count.merge(depend_df, on="product", how="left")
    depend_df["most_depend_perc"] = depend_df["most_depend"] / depend_df["trans_num"] * 100
    
    return depend_df

In [17]:
# (R) Calculating trip driving product dependability metric for each customer segment.
tp_grp1 = trip_driving(df_grp1)
tp_grp2 = trip_driving(df_grp2)
tp_grp3 = trip_driving(df_grp3)

# Select only the products that have been bought multiple times
tp_grp1_top = tp_grp1.query("trans_num >= 50")
tp_grp2_top = tp_grp2.query("trans_num >= 200")
tp_grp3_top = tp_grp3.query("trans_num >= 200")

In [18]:
# (R) Identify trip driving products for each customer segment.
crules1.sort_values("lift", ascending=False)
tp_grp1_top = tp_grp1_top.sort_values("most_depend_perc", ascending=False)
tp_grp1_prod = ["Narcissus Can Food - Whole Mushrooms",
                "Maggie 2-Minute Instant Noodle - Curry",
                "Pasar Malaysia Seedless Guava"]

crules2.sort_values("lift", ascending=False)
tp_grp2_top = tp_grp2_top.sort_values("most_depend_perc", ascending=False)
tp_grp2_prod = ["Maling Premium Luncheon Meat - Pork", 
                "Pasar Long Bean",
                "Pasar Malaysia Seedless Guava"]

crules3.sort_values("lift", ascending=False)
tp_grp3_top = tp_grp3_top.sort_values("most_depend_perc", ascending=False)
tp_grp3_prod = ["Pasar Long Bean",
                "Chef China Garlic - Pure White",
                "Maggi 2-Minute Instant Noodles - Asam Laksa"]

In [307]:
# Create data frame for trip driving barplot
tp1_bar_df = tp_grp1_top[tp_grp1_top["product"].isin(tp_grp1_prod)]
tp1_bar_df["cust_segm2"] = "grp1"
tp2_bar_df = tp_grp2_top[tp_grp2_top["product"].isin(tp_grp2_prod)]
tp2_bar_df["cust_segm2"] = "grp2"
tp3_bar_df = tp_grp3_top[tp_grp3_top["product"].isin(tp_grp3_prod)]
tp3_bar_df["cust_segm2"] = "grp3"

tp_bar_df = (pd.concat([tp1_bar_df, tp2_bar_df, tp3_bar_df])
               .set_index(pd.Index(["0", "1", "2", "3", "4", "5", "6", "7", "8"])))
tp_bar_df["most_depend_perc_str"] = round(tp_bar_df["most_depend_perc"], 1).astype("str")+"%"

In [ ]:
# Barplot for transaction number
ax = sns.barplot(data=tp_bar_df, x="trans_num", y=tp_bar_df.index, color="#0C3578");
ax.set_yticklabels(tp_bar_df["product"]);

sns.set(rc={"figure.facecolor": "#F8F8F8",
            "figure.figsize": (2, 2)})

ax.xaxis.label.set_visible(False);
ax.yaxis.label.set_visible(False);

ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

ax.xaxis.grid(False)
# ax.xaxis.set_label_position("top")
#ax.set_xticklabels(ax.get_xticks(), family="serif", size=8)
ax.set_yticklabels(ax.get_yticklabels(), family="serif", size=8)
ax.set_xticklabels([])
ax.tick_params(bottom=False)

ax.set_xlim(0, 1100)
    
ax.set_facecolor("#F8F8F8")

for patch, perc in zip(ax.patches, tp_bar_df["trans_num"]):
    ax.annotate(perc, 
                (patch.get_width(), patch.get_y()),
                ha="right", va="top",
                xytext=(patch.get_width()-15,patch.get_y()+0.2),
                size=6,
                family="serif",
                color="white")

In [ ]:
# Barplot for most dependable percentage
ax = sns.barplot(data=tp_bar_df, x="most_depend_perc", y=tp_bar_df.index, color="#0C3578");
ax.set_yticklabels(tp_bar_df["product"]);

sns.set(rc={"figure.facecolor": "#F8F8F8",
            "figure.figsize": (2, 2)})

ax.xaxis.label.set_visible(False);
ax.yaxis.label.set_visible(False);

ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

ax.xaxis.grid(False)
# ax.xaxis.set_label_position("top")
#ax.set_xticklabels(ax.get_xticks(), family="serif", size=8)
# ax.set_yticklabels(ax.get_yticklabels(), family="serif", size=8)
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.tick_params(bottom=False)

ax.set_xlim(0, 100)
    
ax.set_facecolor("#F8F8F8")

for patch, perc in zip(ax.patches, tp_bar_df["most_depend_perc_str"]):
    ax.annotate(perc, 
                (patch.get_width(), patch.get_y()),
                ha="right", va="top",
                xytext=(patch.get_width()-2,patch.get_y()+0.2),
                size=6,
                family="serif",
                color="white")

# Impact Estimation

In [19]:
# (R) Narrowing consequents to just antecedent trip driving products
pbundle1 = crules1[crules1["antecedents"].isin(tp_grp1_prod)]
pbundle2 = crules2[crules2["antecedents"].isin(tp_grp2_prod)]
pbundle3 = crules3[crules3["antecedents"].isin(tp_grp3_prod)]

In [20]:
# (R) Creating product bundle dictionary
cgrp1_pbundle = {"Narcissus Can Food - Whole Mushrooms": ["China Shanghai Green Baby", 
                                                          "Lipton Yellow Label Tea Bags - International Blend", 
                                                          "Nissin Instant Noodle - Sesame Oil"],
                 "Maggie 2-Minute Instant Noodle - Curry": ["Narcissus Can Food - Braised Peanuts", 
                                                            "Milo Chocolate Malt Drink Powder with Milk - Australian Recipe", 
                                                            "Narcissus Can Food - Whole Mushrooms"],
                 "Pasar Malaysia Seedless Guava": ["Milo Chocolate Malt Drink Powder with Milk - Australian Recipe", 
                                                   "Narcissus Can Food - Whole Mushrooms"]}

cgrp2_pbundle = {"Maling Premium Luncheon Meat - Pork": ["Chef China Garlic - Pure White"],
                 "Pasar Long Bean": ["FairPrice Thai Hom Mali Superior Fragrant Rice", 
                                     "Lee Kum Kee Oyster Sauce - Panda Brand (770g)", 
                                     "Jing Li Hwa Frozen Fish Ball - Roe"],
                 "Pasar Malaysia Seedless Guava": ["Lee Kum Kee Oyster Sauce - Panda Brand (770g)", 
                                                   "Pasar Long Bean"]}

cgrp3_pbundle = {"Pasar Long Bean": ["Lee Kum Kee Oyster Sauce - Panda Brand (770g)", 
                                     "FairPrice Thai Hom Mali Superior Fragrant Rice", 
                                     "Jing Li Hwa Frozen Fish Ball - Roe"],
                 "Chef China Garlic - Pure White": ["Softy Long & Wide Pantiliners - Fit Absorb (Anti Bacterial)", 
                                                    "Heinz ABC Sweet Sauce", 
                                                    "Lee Kum Kee Oyster Sauce - Panda Brand (770g)"],
                 "Maggi 2-Minute Instant Noodles - Asam Laksa": ["Magiclean Glass Cleaner"]}

In [21]:
# (R) Define function to calculate impact estimation on recommendation of product bundle.
def sales_diff(product_dict, dataframe, impact_goal=0.3, discount=0.1):
    
    # Create a counter for appending data into dataframe
    counter = 0
    new_dataframe = pd.DataFrame(columns=["antecedent", "consequent", 
                                          "antecedent_quan", "consequent_quan",
                                          "diff_quan", "current_sales", 
                                          "add_sales", "add_sales_low", "add_sales_upp"])
    
    for ante, conse_list in product_dict.items():
        
        # Calculate trip driving (antecedent) product sales and quantity
        ante_sales = dataframe[dataframe["product"] == ante]["total_sales"].sum()
        ante_quan = dataframe[dataframe["product"] == ante]["net_quantity"].sum()
        
        # Calculate consequent product sales and quantity
        for conse in conse_list:
            
            # Filter dataframe containing all transactions that contain antecedent product
            receipt_w_ante = dataframe[dataframe["product"] == ante]["receipt_num"]
            df_w_ante = dataframe[dataframe["receipt_num"].isin(receipt_w_ante)]
            
            # Consequent product sales and quantity
            conse_sales = df_w_ante[df_w_ante["product"] == conse]["total_sales"].sum()
            conse_quan = df_w_ante[df_w_ante["product"] == conse]["net_quantity"].sum()
            
            # Calculate difference in sales and quantity of antecedent and consequent products
            diff_quan = round((ante_quan - conse_quan) * impact_goal)
            conse_price = pricing[pricing["product"] == conse].iloc[0, 1]
            add_sales = round(conse_price * diff_quan * (1 - discount))
            add_sales_low = round(conse_price * diff_quan * 0.05 * (1 - discount))
            add_sales_upp = round(conse_price * diff_quan * 0.05 * (1 - discount))
            
            # Calculate current sales for antecedent+consequent pair
            current_sales = round(ante_sales + conse_sales)
            
            new_row = [ante, conse, ante_quan, conse_quan, diff_quan, current_sales, add_sales, add_sales_low, add_sales_upp]
            
            # Append data into dataframe
            new_dataframe.loc[counter] = new_row # type: ignore
            
            counter += 1
            
    return new_dataframe

In [22]:
# (R) Calculate impact estimation for different customer segments
cgrp1_impact = sales_diff(cgrp1_pbundle, df_grp1, impact_goal=0.4, discount=0.1)
cgrp2_impact = sales_diff(cgrp2_pbundle, df_grp2, impact_goal=0.2, discount=0.1)
cgrp3_impact = sales_diff(cgrp3_pbundle, df_grp3, impact_goal=0.1, discount=0.1)